In [1]:
from uncertainties import ufloat
from uncertainties.umath import *
import math
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from astropy.stats import LombScargle
from scipy import signal
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

In [2]:
import glob
from astropy.table import Table,join,vstack,unique

#load in names of OGLE sources in each of II,III,IV that is in part II (no known pulsations)
og2 = Table.read('smc_list/PART2_OII.list',names=['src_n','name','RA','dec'],format='ascii')
og3 = Table.read('smc_list/PART2_OIII.list',names=['src_n','name','RA','dec'],format='ascii')
og4 = Table.read('smc_list/PART2_OIV.list',names=['src_n','name','RA','dec'],format='ascii')

In [4]:
og2[:4]

src_n,name,RA,dec
str4,str14,str10,str11
19,smc_sc5.65500,0:49:11.45,-72:49:37.1
42,smc_sc4.116980,0:47:24.17,-73:12:26.6
61,smc_sc5.111500,0:49:29.81,-73:10:58.0
67,smc_sc2.79541,0:42:07.87,-73:45:01.9


In [5]:
#only keep ints but remember to come back for these sources
og2n = np.array(og2['src_n'])
int_inds2 = [0] #indices of integer src numbers
# c = 0
for i in range(1,len(og2n)):
    if len(og2n[i])>len(og2n[i-1]) and len(og2n[i])>len(og2n[i+1]):
        print(og2n[i])
    else: int_inds2.append(i)

72a
121a


In [6]:
#repeat for OIII and OIV
og3n = np.array(og3['src_n'])
int_inds3 = [0] #indices of integer src numbers
for i in range(1,len(og3n)):
    if len(og3n[i])>len(og3n[i-1]) and len(og3n[i])>len(og3n[i+1]):
        print(og3n[i])
    else: int_inds3.append(i)
        
og4n = np.array(og4['src_n'])
int_inds4 = [0] #indices of integer src numbers
for i in range(1,len(og4n)):
    if len(og4n[i])>len(og4n[i-1]) and len(og4n[i])>len(og4n[i+1]):
        print(og4n[i])
    else: int_inds4.append(i)

105a
114a
67b
68a
116a
132a


In [7]:
#filtered tables
og2f = og2[int_inds2]
og3f = og3[int_inds3]
og4f = og4[int_inds4]

#set data types
og2f = Table(og2f,dtype=('int32','str','str','str'))
og3f = Table(og3f,dtype=('int32','str','str','str'))
og4f = Table(og4f,dtype=('int32','str','str','str'))

In [9]:
#make table with all sources and file names will be added
allog = vstack([og2f,og3f,og4f])
#only keep unique src numbers
allog = unique(allog,keys='src_n')

In [11]:
allog[:3]

src_n,name,RA,dec
int32,str15,str10,str11
1,smc116.2.6392,1:17:05.14,-73:26:36.0
8,smc108.8.124,0:54:10.27,-72:25:44.1
19,smc_sc5.65500,0:49:11.45,-72:49:37.1


In [34]:
#cross tables give V file names based on I file names
cross3 = Table.read('smc_list/IV_OIII.cross',format='ascii',names=['Iname','Vname'])
cross4 = Table.read('smc_list/IV_OIV.cross',format='ascii',names=['Iname','Vname'])

#dictionaries for file names -- will be added to table later
o2idict = dict.fromkeys(allog['src_n'], '')
o2vdict = dict.fromkeys(allog['src_n'], '')
o3idict = dict.fromkeys(allog['src_n'], '')
o3vdict = dict.fromkeys(allog['src_n'], '')
o4idict = dict.fromkeys(allog['src_n'], '')
o4vdict = dict.fromkeys(allog['src_n'], '')

#OII directory and files
i2dir = 'smc_list/OGLE-II/I/'
v2dir = 'smc_list/OGLE-II/V/'
i2f = glob.glob(i2dir+'*')
v2f = glob.glob(v2dir+'*')
#get directories and files
i3dir = 'smc_list/OGLE-III/I/'
v3dir = 'smc_list/OGLE-III/V/'
i3f = glob.glob(i3dir+'*')
v3f = glob.glob(v3dir+'*')
#directories and files for og4: use calibrated
i4dir = 'smc_list/OGLE-IV/I/CALIBRATED/'
v4dir = 'smc_list/OGLE-IV/V/CALIBRATED/'
i4f = glob.glob(i4dir+'*')
v4f = glob.glob(v4dir+'*')

In [35]:
#procedure below effective for all of OII (with integer source numbers)
for s in og2f['src_n']:
    #get source names from og2 table
    #I band name
    name = og2f[og2f['src_n']==s]['name'][0]

    if i2dir+name+'.dat' in i2f:
        print(s,'I found with name')
        o2idict[s] = i2dir+name+'.dat'
        i2f.remove(i2dir+name+'.dat')
    else: print(s,'source not found in I folder')
    
    #repeat with V band
    if v2dir+name+'.dat' in v2f:
        print(s,'V found with name')
        o2vdict[s] = v2dir+name+'.dat'
        v2f.remove(v2dir+name+'.dat')
    else: print(s,'source not found in V folder')

19 I found with name
19 V found with name
42 I found with name
42 V found with name
61 I found with name
61 V found with name
67 I found with name
67 V found with name
68 I found with name
68 V found with name
71 I found with name
71 V found with name
72 I found with name
72 V found with name
73 I found with name
73 V found with name
75 I found with name
75 V found with name
76 I found with name
76 V found with name
77 I found with name
77 V found with name
78 I found with name
78 V found with name
79 I found with name
79 V found with name
80 I found with name
80 V found with name
81 I found with name
81 V found with name
82 I found with name
82 V found with name
84 I found with name
84 V found with name
85 I found with name
85 V found with name
86 I found with name
86 V found with name
87 I found with name
87 V found with name
97 I found with name
97 V found with name
98 I found with name
98 V found with name
101 I found with name
101 V found with name
104 I found with name
104 V foun

In [36]:
for s in og3f['src_n']:
    #get source names from og2 table
    #I band name
    name = og3f[og3f['src_n']==s]['name'][0]
    #corresponding V band name from cross
    if name in cross3['Iname']:
        crname = cross3[cross3['Iname']==name]['Vname'][0]
    else: crname = name
    if v3dir+crname+'.dat' in v3f:
        #add to dictionary
        o3vdict[s] = v3dir+crname+'.dat'
        print(s,'V found with cross name')
        #delete string from v2f
        v3f.remove(v3dir+crname+'.dat')
    else: print(s,'source not found in V folder')
        
    if i3dir+name+'.dat' in i3f:
        print(s,'I found with name')
        o3idict[s] = i3dir+name+'.dat'
        i3f.remove(i3dir+name+'.dat')

    else: print(s,'source not found in I folder')

1 V found with cross name
1 I found with name
8 V found with cross name
8 I found with name
19 V found with cross name
19 I found with name
61 V found with cross name
61 I found with name
67 V found with cross name
67 I found with name
68 V found with cross name
68 I found with name
69 V found with cross name
69 I found with name
71 V found with cross name
71 I found with name
72 V found with cross name
72 I found with name
73 V found with cross name
73 I found with name
75 V found with cross name
75 I found with name
76 V found with cross name
76 I found with name
77 source not found in V folder
77 I found with name
78 source not found in V folder
78 I found with name
79 V found with cross name
79 I found with name
80 V found with cross name
80 I found with name
81 V found with cross name
81 I found with name
82 source not found in V folder
82 I found with name
84 V found with cross name
84 I found with name
85 V found with cross name
85 I found with name
86 V found with cross name
86

In [37]:
for s in og4f['src_n']:
    #get source names from og2 table
    #I band name
    name = og4f[og4f['src_n']==s]['name'][0]
    #corresponding V band name from cross
    if name in cross4['Iname']:
        crname = cross4[cross4['Iname']==name]['Vname'][0]
    else: crname = name
    if v4dir+crname+'.dat' in v4f:
        #add to dictionary
        o4vdict[s] = v4dir+crname+'.dat'
        print(s,'V found with cross name')
        #delete string from v2f
        v4f.remove(v4dir+crname+'.dat')
    else: print(s,'source not found in V folder')
        
    if i4dir+name+'.dat' in i4f:
        print(s,'I found with name')
        o4idict[s] = i4dir+name+'.dat'
        i4f.remove(i4dir+name+'.dat')

    else: print(s,'source not found in I folder')

8 V found with cross name
8 I found with name
68 V found with cross name
68 I found with name
69 V found with cross name
69 I found with name
71 V found with cross name
71 I found with name
72 V found with cross name
72 I found with name
73 V found with cross name
73 I found with name
75 V found with cross name
75 I found with name
76 V found with cross name
76 I found with name
77 source not found in V folder
77 I found with name
78 source not found in V folder
78 I found with name
79 V found with cross name
79 I found with name
80 V found with cross name
80 I found with name
81 V found with cross name
81 I found with name
82 V found with cross name
82 I found with name
84 V found with cross name
84 I found with name
85 V found with cross name
85 I found with name
86 V found with cross name
86 I found with name
87 V found with cross name
87 I found with name
93 V found with cross name
93 I found with name
95 V found with cross name
95 I found with name
96 V found with cross name
96 I 

In [38]:
allog['OII I'] = list(o2idict.values())
allog['OII V'] = list(o2vdict.values())
allog['OIII I'] = list(o3idict.values())
allog['OIII V'] = list(o3vdict.values())
allog['OIV I'] = list(o4idict.values())
allog['OIV V'] = list(o4vdict.values())

In [40]:
allog[:5]

src_n,name,RA,dec,OII I,OII V,OIII I,OIII V,OIV I,OIV V
int32,str15,str10,str11,str37,str37,str38,str38,str49,str49
1,smc116.2.6392,1:17:05.14,-73:26:36.0,,,smc_list/OGLE-III/I/smc116.2.6392.dat,smc_list/OGLE-III/V/smc116.2.5004.dat,,
8,smc108.8.124,0:54:10.27,-72:25:44.1,,,smc_list/OGLE-III/I/smc108.8.124.dat,smc_list/OGLE-III/V/smc108.8.287.dat,smc_list/OGLE-IV/I/CALIBRATED/smc719.19.388.dat,smc_list/OGLE-IV/V/CALIBRATED/smc719.19.2181.dat
19,smc_sc5.65500,0:49:11.45,-72:49:37.1,smc_list/OGLE-II/I/smc_sc5.65500.dat,smc_list/OGLE-II/V/smc_sc5.65500.dat,smc_list/OGLE-III/I/smc101.8.21127.dat,smc_list/OGLE-III/V/smc101.8.29697.dat,,
42,smc_sc4.116980,0:47:24.17,-73:12:26.6,smc_list/OGLE-II/I/smc_sc4.116980.dat,smc_list/OGLE-II/V/smc_sc4.116980.dat,,,,
61,smc_sc5.111500,0:49:29.81,-73:10:58.0,smc_list/OGLE-II/I/smc_sc5.111500.dat,smc_list/OGLE-II/V/smc_sc5.111500.dat,smc_list/OGLE-III/I/smc100.7.58770.dat,smc_list/OGLE-III/V/smc100.7.78408.dat,,


In [43]:
from astropy.io import ascii
ascii.write(allog,'part2files.csv',format='csv')